# Import Libraries

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('diabetes.csv')

In [5]:
X = df[['DiabetesPedigreeFunction', 'Glucose', 'BMI', 'Age']]
y = df['Outcome']

In [7]:
X.head()

,DiabetesPedigreeFunction,Glucose,BMI,Age
0,0.627,148,33.6,50
1,0.351,85,26.6,31
2,0.672,183,23.3,32
3,0.167,89,28.1,21
4,2.288,137,43.1,33


In [8]:
X.shape

(768, 4)

In [9]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [10]:
y.shape

(768,)

***

# Data Preprocessing

In [12]:
from sklearn.preprocessing import StandardScaler

In [18]:
X_sc = StandardScaler()

In [19]:
X_std = X_sc.fit_transform(X)

In [20]:
X_std = pd.DataFrame(data=X_std, columns=X.columns)
X_std.head()

,DiabetesPedigreeFunction,Glucose,BMI,Age
0,0.468492,0.848324,0.204013,1.425995
1,-0.365061,-1.123396,-0.684422,-0.190672
2,0.604397,1.943724,-1.103255,-0.105584
3,-0.920763,-0.998208,-0.494043,-1.041549
4,5.484909,0.504055,1.409746,-0.020496


***

# Devide Train test Split

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.2, random_state=42)

***

# Model Performance

## Performance Function

In [25]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

In [26]:
def print_score(clf, X_train, X_test, y_train, y_test, train = True):
#     print accuracy score, classification report, confusion metrics
    if train:
#         training performance
        print('Train Result : \n')
        print('Accuracy Score {0:.4f}\n'.format(accuracy_score(y_train, clf.predict(X_train))))
        print('Classification Report : \n {} \n'.format(classification_report(y_train, clf.predict(X_train))))
        print('Confusion Metrics : \n {} \n'.format(confusion_matrix(y_train, clf.predict(X_train))))
        
        res = cross_val_score(clf, X_train, y_train, cv = 10, scoring='accuracy')
        print('Average Accuracy : {0:.4f}\n'.format(np.mean(res)))
        print('Accuracy SD : {0:.4f}\n'.format(np.std(res)))
        
    elif train == False:
#         test performance
        print('Test Result : \n')
        print('Accuracy Score {0:.4f}\n'.format(accuracy_score(y_test, clf.predict(X_test))))
        print('Classification Report : \n {}\n'.format(classification_report(y_test, clf.predict(X_test))))
        print('Confusion Metrics : \n {} \n'.format(confusion_matrix(y_test, clf.predict(X_test))))
        

## Different models

### SVM

In [27]:
from sklearn.svm import SVC

In [29]:
clf = SVC(kernel='rbf', C=0.1)

In [ ]:
clf.fit(X)